## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import gmaps
import os

# Import API key
import sys

sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..","Weather_Database","WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Esmeraldas,BR,-19.7625,-44.3139,67.06,92,100,5.88,light rain
1,1,Ushuaia,AR,-54.8000,-68.3000,44.22,75,75,23.02,light intensity drizzle
2,2,Kapaa,US,22.0752,-159.3190,84.18,71,20,4.00,light rain
3,3,Vaini,TO,-21.2000,-175.2000,77.16,83,75,9.22,broken clouds
4,4,Atuona,PF,-9.8000,-139.0333,78.39,76,27,19.48,scattered clouds


In [12]:
# 2. Prompt the user to enter minimum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

#prompt the user to enter maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Esmeraldas,BR,-19.7625,-44.3139,67.06,92,100,5.88,light rain
2,2,Kapaa,US,22.0752,-159.3190,84.18,71,20,4.00,light rain
3,3,Vaini,TO,-21.2000,-175.2000,77.16,83,75,9.22,broken clouds
4,4,Atuona,PF,-9.8000,-139.0333,78.39,76,27,19.48,scattered clouds
5,5,Saint-Philippe,RE,-21.3585,55.7679,71.15,88,6,5.32,clear sky
11,11,Hermanus,ZA,-34.4187,19.2345,63.90,62,98,7.49,overcast clouds
12,12,Mareeba,AU,-17.0000,145.4333,84.33,64,18,1.95,few clouds
13,13,Vao,NC,-22.6667,167.4833,70.48,63,4,13.27,clear sky
14,14,Teluk Intan,MY,4.0259,101.0213,77.22,87,98,2.01,overcast clouds
16,16,Mahebourg,MU,-20.4081,57.7000,71.26,91,96,8.90,light rain


In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                412
City                   412
Country                409
Lat                    412
Lng                    412
Max Temp               412
Humidity               412
Cloudiness             412
Wind Speed             412
Current Description    412
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                409
City                   409
Country                409
Lat                    409
Lng                    409
Max Temp               409
Humidity               409
Cloudiness             409
Wind Speed             409
Current Description    409
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Esmeraldas,BR,67.06,light rain,-19.7625,-44.3139,
2,Kapaa,US,84.18,light rain,22.0752,-159.3190,
3,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
4,Atuona,PF,78.39,scattered clouds,-9.8000,-139.0333,
5,Saint-Philippe,RE,71.15,clear sky,-21.3585,55.7679,
11,Hermanus,ZA,63.90,overcast clouds,-34.4187,19.2345,
12,Mareeba,AU,84.33,few clouds,-17.0000,145.4333,
13,Vao,NC,70.48,clear sky,-22.6667,167.4833,
14,Teluk Intan,MY,77.22,overcast clouds,4.0259,101.0213,
16,Mahebourg,MU,71.26,light rain,-20.4081,57.7000,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
# Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head()
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Esmeraldas,BR,67.06,light rain,-19.7625,-44.3139,Bom Será - Marcinho e Fernanda
2,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Atuona,PF,78.39,scattered clouds,-9.8000,-139.0333,Villa Enata
5,Saint-Philippe,RE,71.15,clear sky,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [18]:
hotel_df.count()

City                   409
Country                409
Max Temp               409
Current Description    409
Lat                    409
Lng                    409
Hotel Name             409
dtype: int64

In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna(axis="index", how="any")
clean_hotel_df.count()

City                   409
Country                409
Max Temp               409
Current Description    409
Lat                    409
Lng                    409
Hotel Name             409
dtype: int64

In [20]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Esmeraldas,BR,67.06,light rain,-19.7625,-44.3139,Bom Será - Marcinho e Fernanda
2,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
4,Atuona,PF,78.39,scattered clouds,-9.8000,-139.0333,Villa Enata
5,Saint-Philippe,RE,71.15,clear sky,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
11,Hermanus,ZA,63.90,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
12,Mareeba,AU,84.33,few clouds,-17.0000,145.4333,Trinity Plains Tourist Park
13,Vao,NC,70.48,clear sky,-22.6667,167.4833,Hotel Kou-Bugny
14,Teluk Intan,MY,77.22,overcast clouds,4.0259,101.0213,Angsoka Hotel Teluk Intan
16,Mahebourg,MU,71.26,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [21]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))